In [1]:
import torch
import numpy as np
import random
import torch.optim as optim

from algorithms import *
from models import *
from dataloaders import *

from tabulate import tabulate

In [2]:
device = 'cpu'
dataset = SyntheticDataset(device=device)

In [3]:
def lr_kde_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_kde_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            h=hp['h'], 
            delta_huber=hp['delta_huber'], 
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test


def experiment_runner(dataset, SGD_hp, EI_hp, DP_hp, EO_hp, EODD_hp, seeds):
    
    _, _, SGD = lr_kde_model_runner(dataset, SGD_hp, seeds)
    _, _, EI = lr_kde_model_runner(dataset, EI_hp, seeds)
    _, _, DP = lr_kde_model_runner(dataset, DP_hp, seeds)
    _, _, EO = lr_kde_model_runner(dataset, EO_hp, seeds)
    _, _, EODD = lr_kde_model_runner(dataset, EODD_hp, seeds)
    
    return SGD, EI, DP, EO, EODD

def hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_kde_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

#### SGD Hyperparameter Selection

In [4]:
SGD_hp_test = {}
SGD_hp_test['learning_rate'] = [0.0001, 0.001, 0.01, 0.1]
SGD_hp_test['lambda_'] = [0]
SGD_hp_test['n_epochs'] = 100
SGD_hp_test['batch_size'] = 1024
SGD_hp_test['fairness'] = ''
SGD_hp_test['h'] = 0.01
SGD_hp_test['delta_huber'] = 0.5
SGD_hp_test['delta_effort'] = 0.5

hyperparameter_test(dataset, SGD_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:05<00:00, 19.52epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:04<00:00, 20.25epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:04<00:00, 20.18epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:05<00:00, 19.93epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val        ei         dp          eo        eodd
---------------  ---------  ----------------  --------------  --------  ---------  ----------  ----------
         0.0001          0          0.794219        0.798438  0.175972  0.0568639  0.359984    0.359984
         0.001           0          0.972187        0.969063  0.197906  0.121821   0.00340553  0.0500687
         0.01            0          0.997188        0.9975    0.135307  0.101419   0.00158604  0.00414326
         0.1             0          0.997969        0.998125  0.114007  0.100794   0.00158604  0.00276217


The learning rate is decided as 0.1.

In [5]:
SGD_hp = SGD_hp_test.copy()
SGD_hp['learning_rate'] = 0.1
SGD_hp['lambda_'] = 0

#### EI Hyperparameter Selection

In [6]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.1]
EI_hp_test['lambda_'] = [0, 0.2, 0.5, 0.7, 0.8, 0.85]
EI_hp_test['fairness'] = 'EI'

hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:05<00:00, 16.93epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:05<00:00, 16.98epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:05<00:00, 16.98epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:05<00:00, 16.69epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei         dp          eo        eodd
---------------  ---------  ----------------  --------------  ---------  ---------  ----------  ----------
            0.1       0             0.997969        0.998125  0.114007   0.100794   0.00158604  0.00276217
            0.1       0.2           0.998281        0.99875   0.0956689  0.0989194  0.00317209  0.00317209
            0.1       0.5           0.997266        0.997188  0.0689833  0.0973569  0.00713719  0.00713719
            0.1       0.7           0.998125        0.997812  0.0760461  0.0979819  0.00555115  0.00555115
            0.1       0.8           0.997344        0.9975    0.0245797  0.0976694  0.00634417  0.00634417
            0.1       0.85          0.751953        0.749687  0.052581   0.148871   0.630881    0.630881


The lambda for EI decided as 0.8.

In [7]:
EI_hp = EI_hp_test.copy()
EI_hp['learning_rate'] = 0.1
EI_hp['lambda_'] = 0.8

#### DP Hyperparameter Selection

In [8]:
DP_hp_test = SGD_hp_test.copy()
DP_hp_test['learning_rate'] = [0.1]
DP_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.9, 0.95]
DP_hp_test['fairness'] = 'DP'

hyperparameter_test(dataset, DP_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:05<00:00, 17.56epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:05<00:00, 17.59epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:05<00:00, 17.60epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:05<00:00, 17.54epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei         dp          eo        eodd
---------------  ---------  ----------------  --------------  ---------  ---------  ----------  ----------
            0.1       0             0.997969        0.998125  0.114007   0.100794   0.00158604  0.00276217
            0.1       0.2           0.997969        0.998437  0.114499   0.100482   0.00158604  0.00207163
            0.1       0.6           0.997891        0.99875   0.114298   0.100169   0.00158604  0.00158604
            0.1       0.8           0.998125        0.999062  0.104447   0.0998569  0.00158604  0.00158604
            0.1       0.9           0.998281        0.998437  0.0982408  0.0992319  0.00317209  0.00317209
            0.1       0.95          0.926016        0.928438  0.111711   0.0286069  0.181602    0.181602


The lambda for DP is decided as 0.95.

In [9]:
DP_hp = DP_hp_test.copy()
DP_hp['learning_rate'] = 0.1
DP_hp['lambda_'] = 0.95

#### EO Hyperparameter Selection

In [10]:
EO_hp_test = SGD_hp_test.copy()
EO_hp_test['learning_rate'] = [0.1]
EO_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.9, 0.95]
EO_hp_test['fairness'] = 'EO'

hyperparameter_test(dataset, EO_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:05<00:00, 17.16epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:06<00:00, 16.17epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:05<00:00, 16.96epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:05<00:00, 16.97epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val        ei        dp           eo        eodd
---------------  ---------  ----------------  --------------  --------  --------  -----------  ----------
            0.1       0             0.997969        0.998125  0.114007  0.100794  0.00158604   0.00276217
            0.1       0.2           0.997891        0.998125  0.1147    0.100794  0.00158604   0.00276217
            0.1       0.6           0.997656        0.998125  0.122322  0.100794  0.00158604   0.00276217
            0.1       0.8           0.997422        0.997812  0.128542  0.101732  0.000793021  0.00414326
            0.1       0.9           0.997188        0.9975    0.139415  0.102044  0.000793021  0.0048338
            0.1       0.95          0.996797        0.996875  0.153211  0.102669  0.000793021  0.00621489


The lambda for EO is decided as 0.8.

In [11]:
EO_hp = EO_hp_test.copy()
EO_hp['learning_rate'] = 0.1
EO_hp['lambda_'] = 0.8

#### EODD Hyperparameter Selection

In [12]:
EODD_hp_test = SGD_hp_test.copy()
EODD_hp_test['learning_rate'] = [0.1]
EODD_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.9, 0.95]
EODD_hp_test['fairness'] = 'EODD'

hyperparameter_test(dataset, EODD_hp_test, seed=1)

training seed 1 started


Training: 100%|██████████| 100/100 [00:06<00:00, 15.02epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:06<00:00, 14.91epochs/s]


Training finished for all seeds.
training seed 1 started


...

Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:06<00:00, 15.00epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:06<00:00, 15.04epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val        ei        dp           eo        eodd
---------------  ---------  ----------------  --------------  --------  --------  -----------  ----------
            0.1       0             0.998047        0.998125  0.109151  0.100541  0.000824402  0.00350049
            0.1       0.2           0.997969        0.998125  0.11318   0.100541  0.000824402  0.00350049
            0.1       0.6           0.997734        0.998125  0.120934  0.100541  0.000824402  0.00350049
            0.1       0.8           0.9975          0.9975    0.131263  0.101172  0.000824402  0.00490069
            0.1       0.9           0.997422        0.9975    0.138337  0.101172  0.000824402  0.00490069
            0.1       0.95          0.997188        0.997188  0.146384  0.102117  0            0.00630088


The lambda for EODD is decided as 0.6.

In [13]:
EODD_hp = EODD_hp_test.copy()
EODD_hp['learning_rate'] = 0.1
EODD_hp['lambda_'] = 0.6

#### Model training

In [14]:
seeds = np.arange(1,6)

SGD, EI, DP, EO, EODD = experiment_runner(dataset, SGD_hp, EI_hp, DP_hp, EO_hp, EODD_hp, seeds)

/var/folders/mt/mgpv1mld71dfr_kn_fq57str0000gn/T/ipykernel_18324/2364504375.py:29: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(seeds[i])


training seed 1 started


Training: 100%|██████████| 100/100 [00:04<00:00, 20.09epochs/s]


training seed 2 started


Training: 100%|██████████| 100/100 [00:04<00:00, 20.24epochs/s]


...

training seed 4 started


Training: 100%|██████████| 100/100 [00:06<00:00, 15.00epochs/s]


training seed 5 started


Training: 100%|██████████| 100/100 [00:06<00:00, 14.89epochs/s]

Training finished for all seeds.


In [15]:
result = []
models = ["SGD", "EI", "DP", "EO", "EODD"]
sol = [SGD, EI, DP, EO, EODD]
for i in range(len(models)):
    c = []
    c.append(models[i])
    res = sol[i]
    c.append(res['accuracy_mean'])
    c.append(res['accuracy_var'])
    c.append(res['ei_mean'])
    c.append(res['ei_var'])
    c.append(res['dp_mean'])
    c.append(res['dp_var'])
    c.append(res['eo_mean'])
    c.append(res['eo_var'])
    c.append(res['eodd_mean'])
    c.append(res['eodd_var'])
    result.append(c)

print(tabulate(result, headers=["model","accuracy_mean","accuracy_var","ei_mean","ei_var", "dp_mean", "dp_var","eo_mean","eo_var","eodd_mean","eodd_var"]))

model      accuracy_mean    accuracy_var    ei_mean      ei_var    dp_mean       dp_var      eo_mean       eo_var    eodd_mean     eodd_var
-------  ---------------  --------------  ---------  ----------  ---------  -----------  -----------  -----------  -----------  -----------
SGD              0.9977      0.0001       0.116566   0.00183322  0.0891411  0.0001       0.00187032   0             0.00335105  0.000216197
EI               0.99635     0.00112472   0.0499404  0.0115562   0.0856911  0.00112472   0.00785536   0.00280479    0.00785536  0.00280479
DP               0.9252      0.00427317   0.1124     0.00290959  0.0141512  0.00457305   0.184582     0.0116608     0.184582    0.0116608
EO               0.9971      0.000122474  0.136887   0.00923241  0.0906445  0.00019068   0.000650407  0.000317621   0.00540492  0
EODD             0.99735     0.000122474  0.124719   0.00230292  0.0899911  0.000406202  0.00124688   0.000394299   0.00464823  0.000551196
